# RAG Source Attribution Analysis

This notebook implements document-level source attribution for Retrieval-Augmented Generation (RAG) systems, based on the methodology from "Source Attribution in Retrieval-Augmented Generation" (arXiv:2507.04480).

## Overview

We will:
1. Load query-document datasets from CSV/JSON files
2. Generate target responses Rtarget using all documents
3. Compute utility functions v(S) for document subsets
4. Apply various attribution methods (Shapley values, approximations, baselines)
5. Visualize and compare attribution results


In [2]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath('')))

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

# Set style for visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

# Import our modules
from rag_system import RAGSystem, load_dataset
import attribution_methods as attr


## 1. Setup: Load Model and Dataset


In [3]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Available memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


Using device: cuda
GPU: NVIDIA H100 80GB HBM3
Available memory: 85.02 GB


In [4]:
# Initialize RAG system
print("Initializing RAG system...")
rag = RAGSystem(model_name="meta-llama/Llama-3.2-1B", device=str(device))
print("RAG system ready!")


Initializing RAG system...
Loading tokenizer for meta-llama/Llama-3.2-1B...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading model...
Model loaded successfully on cuda
RAG system ready!


In [ ]:
# Load dataset
dataset_path = "data/complementary.csv"  # Change this to load different datasets
print(f"Loading dataset from {dataset_path}...")
dataset = load_dataset(dataset_path)
print(f"Loaded {len(dataset)} query-document pairs")

# Display sample
if len(dataset) > 0:
    sample = dataset[0]
    print(f"\nSample query: {sample['question']}")
    print(f"Number of documents: {len(sample['documents'])}")
    print(f"Document IDs: {sample['document_ids']}")
    print(f"\nFirst document: {sample['documents'][0][:100]}...")
    print(f"Answer: {sample['answer']}")


Loading dataset from data/complementary.csv...
Loaded 10 query-document pairs

Sample query: What are the two primary materials used to construct a Xylotian 'Sky-Skiff' hull?
Number of documents: 10
Document IDs: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

First document: The lightweight frame of a Xylotian Sky-Skiff is primarily made from hardened 'Aero-Coral'....
Answer: The two primary materials used to construct a Xylotian 'Sky-Skiff' hull are 'Aero-Coral' for the frame and 'Noctilucent Metal' for the cladding.


## 2. Generate Target Responses

For each query-document pair (Q, D), generate Rtarget = LLM(Q, D) using ALL documents.


In [6]:
# Generate target responses for all queries
target_responses = []

print("Generating target responses...")
for i, item in enumerate(dataset):
    print(f"Processing query {i+1}/{len(dataset)}...")
    rtarget = rag.generate_target_response(
        question=item['question'],
        all_documents=item['documents'],
        max_new_tokens=50
    )
    target_responses.append(rtarget)
    print(f"  Generated: {rtarget[:80]}...")

print(f"\nGenerated {len(target_responses)} target responses")


## 3. Compute Attribution Scores

We'll compute attribution scores using multiple methods for comparison.


In [7]:
# Select a query to analyze (you can change this index)
query_idx = 0
item = dataset[query_idx]
target_response = target_responses[query_idx]

print(f"Analyzing query {query_idx + 1}:")
print(f"Question: {item['question']}")
print(f"Number of documents: {len(item['documents'])}")
print(f"Target response: {target_response}")
print(f"Expected answer: {item['answer']}")


Analyzing query 1:
Question: What are the two primary materials used to construct a Xylotian 'Sky-Skiff' hull?
Number of documents: 10
Target response: The primary materials used to construct a Xylotian 'Sky-Skiff' hull are 'Aero-Coral' and 'Noctilucent Metal'.
Expected answer: The two primary materials used to construct a Xylotian 'Sky-Skiff' hull are 'Aero-Coral' for the frame and 'Noctilucent Metal' for the cladding.


In [8]:
# Create utility function for this query
def utility_function(document_subset: List[str]) -> float:
    """Compute utility v(S) for document subset S."""
    return rag.compute_utility(
        question=item['question'],
        document_subset=document_subset,
        target_response=target_response
    )

# Test utility function
print("Testing utility function...")
print(f"Utility with all documents: {utility_function(item['documents']):.4f}")
print(f"Utility with first document only: {utility_function([item['documents'][0]]):.4f}")
print(f"Utility with empty set: {utility_function([]):.4f}")


In [9]:
# Compute Leave-One-Out attribution
print("Computing Leave-One-Out attribution...")
loo_attributions = attr.leave_one_out(item['documents'], utility_function)
print("Leave-One-Out scores:")
for i, doc_id in enumerate(item['document_ids']):
    print(f"  Document {doc_id}: {loo_attributions[i]:.4f}")


In [10]:
# Compute Monte Carlo Shapley (if we have reasonable number of documents)
n_docs = len(item['documents'])
print(f"Computing Monte Carlo Shapley with {n_docs} documents...")

if n_docs <= 10:
    mc_shapley = attr.monte_carlo_shapley(item['documents'], utility_function, num_samples=64)
    print("Monte Carlo Shapley scores:")
    for i, doc_id in enumerate(item['document_ids']):
        print(f"  Document {doc_id}: {mc_shapley[i]:.4f}")
else:
    print(f"Skipping Monte Carlo Shapley - too many documents ({n_docs})")
    mc_shapley = None


In [11]:
# Compute Permutation-based Shapley
print("Computing Permutation-based Shapley...")
perm_shapley = attr.permutation_shapley(item['documents'], utility_function, num_permutations=50)
print("Permutation Shapley scores:")
for i, doc_id in enumerate(item['document_ids']):
    print(f"  Document {doc_id}: {perm_shapley[i]:.4f}")


In [12]:
# Compute Kernel SHAP (if available and documents are reasonable)
if n_docs <= 10:
    print("Computing Kernel SHAP...")
    try:
        kernel_shap = attr.kernel_shap(item['documents'], utility_function, num_samples=64)
        print("Kernel SHAP scores:")
        for i, doc_id in enumerate(item['document_ids']):
            print(f"  Document {doc_id}: {kernel_shap[i]:.4f}")
    except Exception as e:
        print(f"Kernel SHAP failed: {e}")
        kernel_shap = None
else:
    print(f"Skipping Kernel SHAP - too many documents ({n_docs})")
    kernel_shap = None


In [13]:
# Compute Exact Shapley (only for small document sets)
if n_docs <= 5:
    print("Computing Exact Shapley values...")
    try:
        exact_shap = attr.exact_shapley(item['documents'], utility_function)
        print("Exact Shapley scores:")
        for i, doc_id in enumerate(item['document_ids']):
            print(f"  Document {doc_id}: {exact_shap[i]:.4f}")
    except Exception as e:
        print(f"Exact Shapley failed: {e}")
        exact_shap = None
else:
    print(f"Skipping Exact Shapley - requires 2^{n_docs} = {2**n_docs} evaluations")
    exact_shap = None


Skipping Exact Shapley - requires 2^10 = 1024 evaluations


In [14]:
# Compute Attention-based attribution
print("Computing Attention-based attribution...")
try:
    attention_attr = attr.attention_attribution(
        item['documents'],
        item['question'],
        rag.model,
        rag.tokenizer,
        rag.device
    )
    print("Attention-based scores:")
    for i, doc_id in enumerate(item['document_ids']):
        print(f"  Document {doc_id}: {attention_attr[i]:.4f}")
except Exception as e:
    print(f"Attention attribution failed: {e}")
    attention_attr = None


`sdpa` attention does not support `output_attentions=True` or `head_mask`. Please set your attention to `eager` if you want any of these features.


Computing Attention-based attribution...
Attention attribution failed: object of type 'NoneType' has no len()


## 4. Visualize Attribution Results


In [15]:
# Collect all attribution results
attribution_results = {
    'Leave-One-Out': loo_attributions,
    'Permutation Shapley': perm_shapley,
}

if mc_shapley is not None:
    attribution_results['Monte Carlo Shapley'] = mc_shapley
if kernel_shap is not None:
    attribution_results['Kernel SHAP'] = kernel_shap
if exact_shap is not None:
    attribution_results['Exact Shapley'] = exact_shap
if attention_attr is not None:
    attribution_results['Attention-based'] = attention_attr

# Prepare data for visualization
doc_ids = item['document_ids']
n_methods = len(attribution_results)

# Create comparison plot
fig, axes = plt.subplots(1, n_methods, figsize=(6*n_methods, 6))
if n_methods == 1:
    axes = [axes]

for idx, (method_name, attributions) in enumerate(attribution_results.items()):
    ax = axes[idx]
    scores = [attributions[i] for i in range(len(doc_ids))]
    
    # Normalize scores for better visualization
    scores_norm = np.array(scores)
    if scores_norm.max() != scores_norm.min():
        scores_norm = (scores_norm - scores_norm.min()) / (scores_norm.max() - scores_norm.min())
    
    bars = ax.bar(range(len(doc_ids)), scores_norm, color='steelblue', alpha=0.7)
    ax.set_xlabel('Document', fontsize=12)
    ax.set_ylabel('Normalized Attribution Score', fontsize=12)
    ax.set_title(method_name, fontsize=14, fontweight='bold')
    ax.set_xticks(range(len(doc_ids)))
    ax.set_xticklabels(doc_ids)
    ax.grid(axis='y', alpha=0.3)
    
    # Add value labels on bars
    for i, (bar, score) in enumerate(zip(bars, scores)):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{score:.3f}',
                ha='center', va='bottom', fontsize=9)

plt.suptitle(f'Document Attribution Comparison\nQuestion: {item["question"][:60]}...', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()


In [16]:
# Create heatmap comparing all methods
attribution_matrix = []
method_names = []
for method_name, attributions in attribution_results.items():
    scores = [attributions[i] for i in range(len(doc_ids))]
    # Normalize
    scores_norm = np.array(scores)
    if scores_norm.max() != scores_norm.min():
        scores_norm = (scores_norm - scores_norm.min()) / (scores_norm.max() - scores_norm.min())
    attribution_matrix.append(scores_norm)
    method_names.append(method_name)

attribution_matrix = np.array(attribution_matrix)

plt.figure(figsize=(12, 6))
sns.heatmap(
    attribution_matrix,
    xticklabels=doc_ids,
    yticklabels=method_names,
    cmap='RdYlGn',
    cbar_kws={'label': 'Normalized Attribution Score'},
    annot=True,
    fmt='.3f',
    linewidths=0.5
)
plt.xlabel('Documents', fontsize=12)
plt.ylabel('Attribution Method', fontsize=12)
plt.title('Attribution Methods Comparison Heatmap', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


In [17]:
# Rank documents by each method
print("Document Rankings by Attribution Method:")
print("=" * 60)

for method_name, attributions in attribution_results.items():
    # Get sorted indices
    sorted_indices = sorted(range(len(doc_ids)), key=lambda i: attributions[i], reverse=True)
    
    print(f"\n{method_name}:")
    for rank, idx in enumerate(sorted_indices, 1):
        doc_id = doc_ids[idx]
        score = attributions[idx]
        doc_preview = item['documents'][idx][:50] + "..."
        print(f"  {rank}. Document {doc_id} (score: {score:.4f}): {doc_preview}")


In [18]:
# Summary statistics
print("=" * 80)
print("ATTRIBUTION ANALYSIS SUMMARY")
print("=" * 80)
print(f"\nQuery: {item['question']}")
print(f"Target Response: {target_response}")
print(f"Expected Answer: {item['answer']}")
print(f"\nNumber of documents: {len(item['documents'])}")

print("\n" + "-" * 80)
print("Top 3 Most Important Documents by Each Method:")
print("-" * 80)

for method_name, attributions in attribution_results.items():
    sorted_indices = sorted(range(len(doc_ids)), key=lambda i: attributions[i], reverse=True)
    top_3 = sorted_indices[:3]
    
    print(f"\n{method_name}:")
    for rank, idx in enumerate(top_3, 1):
        doc_id = doc_ids[idx]
        score = attributions[idx]
        print(f"  {rank}. Document {doc_id}: {score:.4f}")

print("\n" + "=" * 80)
